In [6]:
airportsPath = "/data/students/bigdata_internet/lab5/airports.csv"
airlinesPath = "/data/students/bigdata_internet/lab5/airlines.csv"
routesPath = "/data/students/bigdata_internet/lab5/routes.csv"

In [7]:
# defining how to read the date with delimeter
airportsDF = spark.read.csv(airportsPath, header=True, inferSchema=True, sep= ',')
airlinesDF = spark.read.csv(airlinesPath, header=True, inferSchema=True, sep= ',')
routesDF = spark.read.csv(routesPath, header=True, inferSchema=True, sep= ',')

In [45]:
# airportsDF.show(2, truncate=False)
# airportsDF.select("*").filter(airportsDF.name.contains("Catania")).show()
# Los Angeles International Airport 3484
# Catania-Fontanaro... 1509

## 2.1

In [8]:
from pyspark.sql.functions import count, asc
# Group the data by country and count the number of airports
grouped_data = airportsDF.groupBy("country").agg(count("*").alias("count"))
# Filter the data to only include countries with more than 200 airports
filtered_data = grouped_data.filter(grouped_data["count"] > 200)
# Show the filtered data
# filtered_data.show()

In [14]:
# countries having more than 200 airports are: 
# +-------------+-----+
# |      country|count|
# +-------------+-----+
# |       Russia|  264|
# |      Germany|  249|
# |       France|  217|
# |        China|  241|
# |United States| 1512|
# |       Canada|  430|
# |       Brazil|  264|
# |    Australia|  334|
# +-------------+-----+

## 2.2

In [261]:
# Group the data by country and find the maximum altitude for each country
# Rename the resulting column from "max(altitude)" to "maxAltitude"
# Sort the data by the maximum altitude in descending order
# Select the top 5 countries with the highest maximum altitude
groupedByCountry = airportsDF.groupBy("country").agg({"altitude": "max"})\
        .withColumnRenamed("max(altitude)", "maxAltitude")\
        .sort("maxAltitude", ascending = False)
metered_df = groupedByCountry.withColumn("altitude", groupedByCountry["maxAltitude"] * 0.3048)
top5 = metered_df.limit(5)

# Show the results
top5.show(5,truncate=False)

In [47]:
# top 5 airports in terms of Altitude :
# +---------+-----------+
# |country  |maxAltitude|
# +---------+-----------+
# |China    |14472      |
# |Bolivia  |13355      |
# |Peru     |12552      |
# |Nepal    |12400      |
# |Argentina|11414      |
# +---------+-----------+

# in Meters
# +---------+-----------+------------------+
# |country  |maxAltitude|altitude          |
# +---------+-----------+------------------+
# |China    |14472      |4411.0656         |
# |Bolivia  |13355      |4070.6040000000003|
# |Peru     |12552      |3825.8496         |
# |Nepal    |12400      |3779.52           |
# |Argentina|11414      |3478.9872         |
# +---------+-----------+------------------+

## 2.3

In [262]:
# first groupby data on "airport_source_id", "airport_destination_id" to get groups of the same src_dest
#order them based on count of routes in descending order
# count each group and put it in a column
grouped_routes = routesDF.groupBy("airport_source_id", "airport_destination_id").agg({"*":"count"})\
                            .orderBy("count(1)", ascending=False)
#join resulted df with airport df to get the source city and airport name
#select needed columns
source_routes = grouped_routes.join(airportsDF, grouped_routes["airport_source_id"] == airportsDF["id"])\
                    .select("airport_source_id","name","city","airport_destination_id","count(1)")\
                    .withColumnRenamed("name","src_name").withColumnRenamed("city","src_city")
#join resulted df with airport df to get the destination city and airport name
#select needed columns
dest_routes = source_routes.join(airportsDF, source_routes["airport_destination_id"] == airportsDF["id"])\
                    .select("airport_source_id","src_city",\
                    "src_name","airport_destination_id","city","name","count(1)")\
                    .withColumnRenamed("name","dest_name").withColumnRenamed("city","dest_city")\
                    .withColumnRenamed("count(1)","routes_count")
dest_routes.show(10)

In [ ]:
# +-----------------+--------------------+-----------+----------------------+-----------+--------------------+------------+
# |airport_source_id|           dest_name|  dest_city|airport_destination_id|  dest_city|           dest_name|routes_count|
# +-----------------+--------------------+-----------+----------------------+-----------+--------------------+------------+
# |             3830|Hartsfield Jackso...|    Atlanta|                  3682|    Atlanta|Hartsfield Jackso...|          20|
# |             3682|Chicago O'Hare In...|    Chicago|                  3830|    Chicago|Chicago O'Hare In...|          19|
# |             3830|Louis Armstrong N...|New Orleans|                  3861|New Orleans|Louis Armstrong N...|          13|
# |             3179|Suvarnabhumi Airport|    Bangkok|                  3885|    Bangkok|Suvarnabhumi Airport|          13|
# |             2179|Muscat Internatio...|     Muscat|                  2194|     Muscat|Muscat Internatio...|          12|
# |             3682|Miami Internation...|      Miami|                  3576|      Miami|Miami Internation...|          12|
# |             3576|Hartsfield Jackso...|    Atlanta|                  3682|    Atlanta|Hartsfield Jackso...|          12|
# |             3885|Hong Kong Interna...|  Hong Kong|                  3077|  Hong Kong|Hong Kong Interna...|          12|
# |             3370|Hangzhou Xiaoshan...|   Hangzhou|                  3386|   Hangzhou|Hangzhou Xiaoshan...|          12|
# |             3797|London Heathrow A...|     London|                   507|     London|London Heathrow A...|          12|
# +-----------------+--------------------+-----------+----------------------+-----------+--------------------+------------+

## 3.1

In [212]:
fltred_routes =routesDF.filter((routesDF["airport_source_id"] != "\\N") \
                               & (routesDF["airport_destination_id"] != "\\N"))
fltred_routes.count()

67240

routesDF count before filtering : 67663

routesDF count after filtering : 67240

423 rows were filtered out !

In [134]:
#vertices only need id and name
#edges need source and destination Ids plus airline Id
vertices = airportsDF.select("id", "name", "city", "country", "icao")\
                        .withColumnRenamed("id", "id")\
                        .withColumnRenamed("name", "label")
edges = fltred_routes.select("airport_source_id", "airport_destination_id", "airline_id")\
                .withColumnRenamed("airport_source_id", "src")\
                .withColumnRenamed("airport_destination_id", "dst")\
                .withColumnRenamed("airline_id", "label")

In [135]:
#converting columns to string
str_vertices = vertices.withColumn("id", vertices.id.cast("string"))
str_edges = edges.withColumn("src", edges.src.cast("string"))\
                .withColumn("dst", edges.dst.cast("string"))\
                .withColumn("label", edges.label.cast("string"))         

In [136]:
from graphframes import GraphFrame
g = GraphFrame(str_vertices, str_edges)

## 4.1

In [139]:
#getting the in_degrees and out_degrees
in_degree = g.inDegrees
out_degree = g.outDegrees

In [140]:
# joining vertices with in_degree df and ou_degree to get the needed table
airports_degree = vertices.join(in_degree, on='id').join(out_degree, on='id')

In [263]:
#addign a column called ration which is indegree/outdegree
#sorting based on ration in descending order then getting only top 5
# out_degree.filter(out_degree.id == 1501).count() ===> 50
from pyspark.sql.functions import col
airports_ratio = airports_degree.withColumn("ratio", col("inDegree") / col("outDegree"))
top_5_airports = airports_ratio.sort(col("ratio").desc()).limit(5)
top_5_airports.show()

In [ ]:
# +----+--------------------+----------+---------------+--------+---------+-----+
# |  id|               label|      city|        country|inDegree|outDegree|ratio|
# +----+--------------------+----------+---------------+--------+---------+-----+
# |3862|Portland Internat...|  Portland|  United States|      18|        2|  9.0|
# | 653|       Banak Airport|   Lakselv|         Norway|       3|        1|  3.0|
# |5416|    Suavanao Airport|  Suavanao|Solomon Islands|       3|        1|  3.0|
# |6289|   Longreach Airport| Longreach|      Australia|       5|        2|  2.5|
# |4285|      Yeager Airport|Charleston|  United States|      15|        6|  2.5|
# +----+--------------------+----------+---------------+--------+---------+-----+

## 4.2.1

In [142]:
#useing the motif setting the destination id 1501 which is bari
bari_dest_motif = g.find("(a)-[e]->(b)").filter("b.city = 'Bari'").filter("b.id = 1501")
bari_dest_motif.count()

50

In [213]:
# out_degree.filter(out_degree.id == 1501).show()
# +----+---------+
# |  id|outDegree|
# +----+---------+
# |1501|       50|
# +----+---------+
# in_degree.filter(in_degree.id == 1501).show()
# +----+--------+
# |  id|inDegree|
# +----+--------+
# |1501|      50|
# +----+--------+

##### from 50 airports you can reach Bari in only one flight
##### in degree is 50 and it is reachable with 50 straight flights

### 4.2.2

In [143]:
#useing the motif setting the source id 1501 which is bari
bari_src_motif = g.find("(a)-[e]->(b)").filter("a.city = 'Bari'").filter("a.id = 1501")
bari_src_motif.count()

50

##### you can reach from Bari to 50 airports in only one flight

### 4.2.3

#### inDegrees equals to outDegrees, the same way that there are 50 flights
#### from airports to Bari there are flights going back to the the same exact airports

### 4.3

In [144]:
#we start from Bari - we reach to an airport - then we move to the next Airport which is our destincation
#there might repeatative flights so we get the distinct
two_edge_motif = g.find("(a)-[e1]->(b); (b)-[e2]->(c)")\
                        .filter("a.city = 'Bari'").filter("a.id = 1501")
two_edge_motif.distinct().count()

7265

#### we can reach 7265 airports from Bari with only two flights

### 4.4.1

In [145]:
#3484 1509
g1 = g.filterVertices("id = 3484 or id = 1509")

In [146]:
display(g1.vertices)

DataFrame[id: string, label: string, city: string, country: string, icao: string]

In [147]:
display(g1.edges)

DataFrame[src: string, dst: string, label: string]

In [264]:
# result = g.bfs(fromExpr="id!=1509 and id!=3484",toExpr="id=1509 or id=3484")
# result.show()
LA_result = g.bfs(fromExpr="id != 3484", toExpr="id=3484")\
                .withColumnRenamed("from","la_from")\
                .withColumnRenamed("e0","la_e0")\
                .withColumnRenamed("to","to_la")
# LA_result.count() ===> 497
CA_result = g.bfs(fromExpr="id != 1509", toExpr="id=1509")\
                .withColumnRenamed("from","ca_from")\
                .withColumnRenamed("e0","ca_e0")\
                .withColumnRenamed("to","to_ca")
# CA_result.count() ======> 118
# temppp = LA_result.join(CA_result)
LA_result.show()
# LA_result.show()
# less_hops_LA = CA_hops > LA_hops
# num_airports = less_hops_LA.filter(less_hops_LA == True).count()

### 4.5

In [266]:
#grouping by component and counting them based in their unique ids
sc.setCheckpointDir("temp_ckpts")
cc = g.connectedComponents().groupBy('component').agg(count('*').alias('count'))
fltrd_cc = cc.filter(cc["count"] >= 2)
# fltrd_cc.show()

In [129]:
print("Number of Connected Components: ",fltrd_cc.distinct().count())

Number of Connected Components:  7


#### Number of Connected Components:  7

In [ ]:
# +-------------+-----+
# |    component|count|
# +-------------+-----+
# | 266287972371|    4|
# | 352187318292|    4|
# | 721554505752|    2|
# |1460288880657|    2|
# | 300647710730|   10|
# |           11| 3188|
# |           23|    4|
# +-------------+-----+

## 4.6

In [267]:
fltrd_arlines= airlinesDF.filter("icao != 'N/A'")
resullt = fltred_routes.join(fltrd_arlines, fltrd_arlines["airline_id"] == fltred_routes["airline_id"])\
                        .drop(fltred_routes["airline_id"])
routes_city_6 = resullt.groupBy(resullt["icao"], resullt["airline_id"])\
                .agg(count(resullt["airport_source_id"]).alias("ID_count"))\
                        .filter("ID_count >= 6")\
                        .sort("ID_count")
oops = routes_city_6.join(fltred_routes, fltred_routes["airline_id"] == routes_city_6["airline_id"])\
                        .drop(fltred_routes["airline_id"])
nice = oops.select(oops["airline_id"],oops["airport_source_id"],\
                         oops["airport_source"],oops["airport_destination"],oops["airport_destination_id"],\
                         oops["icao"],oops["ID_count"])
# nice.show()

In [ ]:

import networkx as nx
def xGraph(edge_list,node_list): Gplot=nx.DiGraph() edges=edge_list.collect() nodes=node_list.collect()

for row in edges: Gplot.add_edge(row['src'],row['dst'])
for row in nodes: Gplot.add_node(row['id'])
return Gplot

Gplot=xGraph(g.edges,g.vertices)
nx.draw(Gplot, with_labels=True, font_weight='bold')